# Advanced Machine Learning - programming assignment 1

*Due: Friday November 29th*

**Please fill in:**
* Cem Kaya (9276866)
* Gaynora van Dommelen (6717659)

### Further instructions:
* Code quality is considered during the assessment. Make sure your code is properly commented. 
* Submit your code in Blackboard using **one** of your accounts; we will put the grade in Blackboard for the other team member as well.
* Make sure to name the submitted file according to your and your collaborators last name (i.e. submitter_collaborator.ipynb). 
* **Failure to follow these instructions can affect the assignment grade.**

In [ ]:
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt

### 0. Please check the given 'README' file

### 1. Let's start with the OpenAI gym

Gym/Gymnasium (https://gymnasium.farama.org/) is a wide-used toolkit for developing and comparing reinforcement learning algorithms. 

1. Gym/Gymnasium makes no assumptions about the structure of your agent, and is compatible with any numerical computation library, such as TensorFlow or Theano. 

2. The library is a collection of test problems — **environments** — that you can use to work out your reinforcement learning algorithms. These environments have a shared interface, allowing you to write general algorithms.

**Great!** Now let's import the gymnasium class and work on a basic example of gym code.


In [ ]:
import gymnasium

Like mentioned above, gymnasium's main purpose is to provide a large collection of **environments** that expose a common interface. You can find a listing of those environments below (they are Markov decision process(MDP) environments and we discussed MDP in our lecture 2), as follows:

In [ ]:
from gymnasium import envs
print(envs.registry)

We are now going to explain how the RL framework of gym works. 
- An **ENVIRONMENT**, 
- You also have an **AGENT**,
- The agent takes an **ACTION**, in our case, 10 actions are possible to take,
- When a single **ACTION** is chosen and fed to our **ENVIRONMENT**, the **ENVIRONMENT** measures how good the action was taken and produces a **REWARD**, which is usually a numeric value.

In MDP problems, the **ENVIRONMENT** will also provides an **OBSERVATION**, which represets the state of the **ENVIRONMENT** at the current moment. In the multi-armed bandit problems, there is no **OBSERVATION** (or state) or one constant state. 

Please read the 'Basic usage' https://gymnasium.farama.org/introduction/basic_usage/ for better understanding the framework. 


### 2. Implement your own environment

Next, we are going to implement our own environment following the framework of gym. This enviroment is a gambiling room with ten different slot machines (a 10-armed bandit problem). Similar with examples given in the lecture, the reward of each slot machine follows a normal distribution, but the average reward (mean) and variance of each action are different. Your goal is to determine the optimal action from all possible actions/machines. 

The core gym interface is **Env**, which is the unified environment interface. There is no interface for agents. The following are the Env methods you should know:

- `step(self, action)`: Steps the environment by one timestep. Returns observation, reward, done, info.
- `reset(self)`: Resets the environment to an initial state. Returns an initial observation. Each call of `reset()` should yield an environment suitable for a new episode, independent of previous episodes. Because there is no state transition in multi-armed bandit problems, this function is not used here.
- `render(self, mode='human')`: Renders one frame of the environment. The default mode will do something human friendly, such as pop up a window. In this assignment, there is no need to create a pop up window. 

Before writing your own codes, read through the readme of github page of gymasium (https://github.com/Farama-Foundation/Gymnasium). You are also recommended to read at least the codes for one simple environment and one example agent.

#### 2.1 Self-defined Slot Machine

**Please fill in the missing codes in the function sample (1 point).**

In [ ]:
import numpy as np

class slotMachine:
    """
        A slot machine contains a reward distribution that randomly generated with restricted mean and standard deviation. 
            sample function: generates a reward at each time step based on the given reward distribition
    """
    def __init__(self):
        self.mu = np.random.uniform(-5, 5)  # mean
        self.sigma = np.random.uniform(0.5, 1)  # standard deviation

    def sample(self):
        ########## TODO: to be filled. ########## 
        return np.random.normal(self.mu, self.sigma)

#### 2.2 Game Environment
**Please fill in the missing codes in function step (1 point) in the environment.** 

In [ ]:
from gymnasium import spaces

# The environment has to inherit the interface of gymnasium.Env
class GamblingRoom(gymnasium.Env):
    """
    A k-armed bandit environment: a gambling room with slot machines, allows the agents to interact with it.
        r_machines: A list of slot machines, each gamblingRoom contains k number of slotMachines
    """
    def __init__(self, k: int, seed: int=None):
        # set random seed
        self.seed(seed)     
        # initialize reward distribution for each action/machine
        self.r_machines: List[slotMachine] = []
        for i in range(k):
            # each gamblingRoom contains k number of slotMachines
            self.r_machines.append(slotMachine())

        self.num_arms: int = k
        self.action_space = spaces.Discrete(self.num_arms)
        self.observation_space = spaces.Discrete(1)
        # for our bandit environment, the state is constant
        self.state: int = 0
    
    # step up the environment based on the selected action,
    # return the constant state, reward, done = false, and info 
    # for now, we do not have to worry about the DONE and INFO variables.
    def step(self, action: int) -> Tuple[int, float, bool, dict]:
        assert self.action_space.contains(action)
        done = False

        ########## TODO: to be filled. ##########
        reward = self.r_machines[action].sample()
        return self.state, reward, done, {}

    # random seed used for reproducibility purposes
    def seed(self, seed: int):
        if seed is not None:
            np.random.seed(seed)
    
    def reset(self):
        pass

    def render(self, mode:str='human', close:bool=False):
        pass

    def close(self):
        pass

### 3. Implement an agent with the epsilon greedy algorithm

In this part, you are expected to implement an RL agent. To decide the action to take at each time step, this agent uses the epsilon greedy algorithm introduced in the lecture.

**Please fill in the missing codes in function select_action (1.5 points) and update_parameters (1 point) in the agent.** Feel free to import the needed packages if there are any.

In [ ]:
class EplisonGreedyAgent:
    def __init__(self, k: int, e: float):
        # set up the number of arms/actions
        self.num_arms: int = k
        # set up the value of epsilon
        self.epsilon: float = e
        # init the estimated values of all actions
        self.Qvalues: np.ndarray = np.zeros(k)
        # init the numbers of time step that every action is selected
        self.stepSize: np.ndarray = np.zeros(k)

    ##
    # select the action to take at the current time step
    # (for MDP, choose the action based on state; for k-armed bandit, no state given)
    # return: the action to take
    ##
    def select_action(self) -> int:
        """
        Select the action to take at the current time step using the epsilon greedy algorithm.

        Returns
        -------
        int
            The index of the selected action
        """
        ########## TODO: to be filled. ##########   
        if np.random.rand() < self.epsilon: # exploration
            return np.random.randint(0, self.num_arms)
        else: # exploitation
            return np.argmax(self.Qvalues) # return index of largest Qvalue

    ##
    # Update the Q-values of the agent based on received rewards
    # input: action_index = the action, reward = the reward from this action
    # return: null
    ##
    def update_parameters(self, action: int, reward: float) -> None:
        """
        Update the number of times an action was taken as well as the running average of the Q-values.

        Parameters
        ----------
        action: int
            The index of the action which needs to be updated
        reward: float
            The reward corresponding to the provided action
        """
        ########## TODO: to be filled. ##########  
        self.stepSize[action] += 1
        self.Qvalues[action] += (reward - self.Qvalues[action]) / self.stepSize[action]
    
        

### 4. Run the simulation, play with parameters and analyse results

Finally, we write codes for running the simulation. 

In order to decrease the effect of randomness, we usually conduct multiple simulation runs and average the results. In the implementation, you may start with one run, then use the variable `num_runs` for running multiple simulations.

In each run, you shall setup the `epsilon` and number of time step `num_episodes` (0.01 and 500 by default). Then, after the initlization of our agent and environment, **please fill in the missing codes (with ??? or TODO: to be filled). (2.5 points)**

In [ ]:
# tested hyper parameters
num_episodes_list = [1000, 2000, 5000]
epsilon_list = [0, 0.01, 0.1]

In [ ]:
def simulate(k: int, seed: int, num_runs: int, num_episodes: int, epsilon: float) -> Tuple[List[float], List[float]]:
    """Simulate multiple runs of the k-bandit problem depicted as a gambling room
    
    Parameters
    ----------
    k: int
        The number of arms (i.e., slot machines)
    seed: int
        Used for reproducibility
    num_runs: int
        The number of iterations the simulation must go through
    num_episodes: int
        The number of steps that the agent must take during each run
    epsilon: float
        The fraction of greediness of the agent
    
    Returns
    -------
    Tuple[List[float], List[float]]
        List of average rewards and list of optimal actions counts
    """
    # init the environment and set up the random seed
    env = GamblingRoom(k=k, seed=seed)

    # delete the wrap
    env = env.unwrapped

    # show the action space
    print(env.action_space)

    # to store average reward at each timestep
    average_rewards = np.zeros(num_episodes)
    # to store the amount of times the optimal action has been chosen
    optimal_action_counts = np.zeros(num_episodes)

    # run simulations
    for i_run in range(num_runs):
        agent = EplisonGreedyAgent(k=k, e=epsilon)

        true_action_values = np.array([machine.mu for machine in env.r_machines])
        optimal_action = np.argmax(true_action_values)
        
        for t in range(num_episodes):
            action = agent.select_action() # make the agent choose its next action
            
            _, reward, _, _ = env.step(action) # make the agent take a step
            
            agent.update_parameters(action, reward) # update the params of the agent based on the achieved rewards
             
            average_rewards[t] += reward # store reward
            
            if action == optimal_action: # if the optimal action was chosen
                optimal_action_counts[t] += 1 # increment the counter for optimal action at time step t
            
    env.close()
    average_rewards /= num_runs
    optimal_action_counts /= num_runs
    return average_rewards, optimal_action_counts

In [ ]:
# storage of results for each huper parameter
results = {}

for num_episodes in num_episodes_list:
    print(f"Running for {num_episodes} episodes")
    for epsilon in epsilon_list:
        
        num_action = 10
        num_seed = 5
        num_runs = 2000  # number of simulation runs
        average_rewards, optimal_action_counts = simulate(k=num_action, seed=num_seed, num_runs=num_runs, num_episodes=num_episodes, epsilon=epsilon)
        
        # Store the results for this combination of (num_episodes, epsilon)
        results[(num_episodes, epsilon)] = {
            "average_rewards": average_rewards,
            "optimal_action_counts": optimal_action_counts,
        }



In [ ]:
def plot_simulation_evaluation(results: Dict[Tuple[int, float], Dict[str, List[float]]]) -> plt.Axes:
    """Plot simulation evaluation for agents with different epsilon values.
    
    Parameters
    ----------
    results: Dict[Tuple[int, float], Dict[str, List[float]]]
        Key is a tuple of the number of episodes and epsilon with values:
        - 'average_rewards': List of average rewards for each episode
        - 'optimal_action_counts': List of counters for the times an optimal action was taken
        
    Returns
    -------
    plt.Axes
        Axes objects of the average rewards over episodes and the percentage of optimal actions chosen over episodes
    """
    # Create figure to store plots
    fig, ax = plt.subplots(1, 2, figsize=(14,6))

    
    for (episodes, epsilon), data in results.items():
        # Plot the average rewards
        ax[0].plot(
            range(episodes),
            data['average_rewards'],
            label=f"$\\varepsilon$={epsilon}"
        )
        # Plot the percentage of optimally chosen actions
        ax[1].plot(
            range(episodes),
            data['optimal_action_counts']*100,
            label=f"$\\varepsilon$={epsilon}"
        )
    # Style the graph of the average reward performance
    ax[0].set_title("Average reward performance")
    ax[0].set_xlabel("Episodes")
    ax[0].set_ylabel("Average reward")
    ax[0].legend(loc="lower right")
    # Style the graph of the percentage optimally chosen actions
    ax[1].set_title("Percentage optimally chosen actions")
    ax[1].set_xlabel("Episodes")
    ax[1].set_ylabel("% Optimal action")
    ax[1].legend(loc="lower right")
    return ax

In [ ]:
get_results_episode = lambda n_episodes: dict((k,v) for k,v in results.items() if k in list(filter(lambda key: n_episodes == key[0], results.keys())))

In [ ]:
ax = plot_simulation_evaluation(get_results_episode(1000))

In [ ]:
ax = plot_simulation_evaluation(get_results_episode(2000))

In [ ]:
ax = plot_simulation_evaluation(get_results_episode(5000))

Now it's time to examine the performance of algorithms with different epsilon values (different exploration strategies) in multiple simulation runs. 

You shall play with the parameter epsilon under 2 or 3 different gambling environments (by initlizing different reward distributions for machines). **For each environment, try at least 2 different values of epsilon and identify a reasonable epsilon value that could balance the exploration and exploiation**. **Instead of handing in your codes for this part, please select one environment you have tested and describe your environment and experimental settings (1 point)**. Then, provide an explanation on how you identify the good epsilon value in this environment and why it is a good one **(1 point)**. 

Few instructions:
- Your answer shall include two plots presenting compariable measures of the different epsilon settings (e.g. the average reward per step and % of optimal action). **(1 point)** 
- You shall present the average results from at least 100 simulation runs. Remember that the gambling environment CANNOT be changed over those runs used for calculating the average results. 
- You may adjust the total time steps when the learning needs more time for a cerain epsilon value, but do not over spend your time on this.    

**Put your answer (at most 300 words) with accompanying plots here.** 

> **CHOSEN ENVIRONMENT AND EXPERIMENTAL SETTINGS**  
> * `num_runs=2000`, to limit the oscillation and variation between episodes due to the number of runs, the number of runs was adjusted to `2000`; copying the number of runs used in the RL book to demonstrate the performance of $\varepsilon$-greediness.
> * `num_episodes_list = [1000, 2000, 5000]`, within the book they mention that an agent using $\varepsilon=0.01$ greediness performs better in the long run than that of $\varepsilon=0.1$. This was, however, not depicted within the graph in the RL book. Therefore, additional number of episodes were chosen to see when the $\varepsilon=0.01$ strategy overtakes $\varepsilon=0.1$, if it happens at all.
> * `epsilon_list = [0, 0.01, 0.1]`, again using the RL book as inspiration, the epsilon values include a fully greedy one exploring to its hearts content and less greedy ones.
>
> **OPTIMAL PERFORMING EPSILON AGENT**  
> As determined within the RL book, the results show that an epsilon-greedy agent with $\varepsilon=0.01$ performs better in the long run compared to $\varepsilon=0.1$. This can be seen when the number of episodes is increased to above $1000$. Then between $2000$ and $3000$ episodes the average reward seems to have reached its optimal and evens out, while the optimally chosen actions by the agent with $\varepsilon=0.01$ only reaches its optimum near $5000$ episodes; effectively overtaking the percentage of optimally chosen actions when an agent uses $\varepsilon=0.1$. Therefore, the best performing epsilon agent uses a $\varepsilon=0.01$ to determine its greediness.
>
> ![image.png](img/epsilon-greedy-agent-episodes-5000.png)

You are almost done! Before handing in, make sure that the codes you hand in work, and that all plots are shown. **Submit just one file per team.** Please make sure that you submit a .zip file with images.

Again, make sure you name this file according to your last names.